In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cleaning the scrape artifacts.

In [2]:
fpl = pd.read_csv('../part-01/fpl_raw_scrape.csv')
shaw = pd.read_csv('../part-01/shaw.csv')
git = pd.read_csv('../part-01/fpl_18_19.csv')

The scrape coudn't pick up shaw due to timeout, so seperatly scraped and will be concatenated during cleaning.

In [3]:
fpl.drop('Unnamed: 0', axis = 1, inplace = True)
shaw.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
fpl['player'].isnull().sum()

70

I think 35 from shaw and another from an empty player.

In [5]:
fpl.dropna(inplace=True, subset=['player'])

In [6]:
fpl.shape

(18603, 29)

In [7]:
shaw.shape

(38, 29)

In [8]:
fpl=pd.concat([fpl,shaw], axis=0)

There are 2 Danny Wards in the PL!!!!

In [9]:
list(fpl[fpl['player']=='Danny Ward']['gw'])

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 32.0,
 32.0,
 34.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0]

When importing git the danny ward that got 19 points was danny ward 1, danny ward 2 got 0.

In [10]:
git[git['player']=='danny ward 1']['total_points'].sum()

19

In [11]:
git[git['player']=='danny ward 2']['total_points'].sum()

0

In [12]:
fpl[fpl['player']=='Danny Ward']['team'].unique()

array(['Cardiff', 'Leicester'], dtype=object)

In [13]:
print(fpl[(fpl['player']=='Danny Ward')&
    (fpl['team']=='Cardiff')]['pts'].sum())
danny_1 = fpl[(fpl['player']=='Danny Ward')&
    (fpl['team']=='Cardiff')].index

19.0


In [14]:
print(fpl[(fpl['player']=='Danny Ward')&
    (fpl['team']=='Leicester')]['pts'].sum())
danny_2 = fpl[(fpl['player']=='Danny Ward')&
    (fpl['team']=='Leicester')].index

0.0


Dannies are now renamed

In [15]:
fpl.loc[danny_1,'player']='Danny Ward 1'
fpl.loc[danny_2,'player']='Danny Ward 2'

Now to clean up some of the columns. Value, has a currency sign while selected, transfers have commas and the opposition values have lots of white space and escape keys.

In [16]:
def value_fix(value):
    if type(value) == float:
        return value
    else:
        return float(value[1:])

def selected_by_fix(selected_by):
    if type(selected_by) == float:
        return selected_by
    else:
        return float(selected_by.replace(',',''))

def net_transfers_fix(net_transfers):
    if type(net_transfers) == float:
        return net_transfers
    else:
        return float(net_transfers.replace(',',''))
    
def opp_split(opp):
    '''
    lists and removes escape keys
    in opp string. Used in extracting
    data from the opp feature. Which can 
    contain opponent, home\away and score.
    '''
    if '\n' in opp:
        wordlist = [word.replace('\n','') for word in opp.split(' ') if (word != '') and (word != '\n')]
        if len(wordlist) == 3:
            wordlist = [wordlist[0]+wordlist[1], wordlist[2]]
        return wordlist
    else:
        other = opp[:7]
        score = opp[8:]
        wordlist = [word for word in other.split(' ')] + [score]
        return wordlist

In [17]:
fpl['value'] = fpl['value'].apply(value_fix)
fpl['selected_by'] = fpl['selected_by'].apply(selected_by_fix)
fpl['net_transfers'] = fpl['net_transfers'].apply(net_transfers_fix)

opposition still has lots of information that should be seperated out into different columns.

In [18]:
def opp_extract_opposition(opp):
    '''
    if opp is not np.nan(float) then 
    take then take first element from 
    listed value.
    '''
    if type(opp) != float:
        opposition = opp_split(opp)[0]
        return opposition
    else:
        return opp

def opp_extract_is_home(opp):
    '''
    if opp is not np.nan(float) then 
    take then take second element. If
    second == H, return 1 else 0.
    '''
    if type(opp) != float:
        is_home = opp_split(opp)[1]
        if is_home == '(H)':
            return 1
        else:
            return 0
    else:
        return opp

def opp_extract_score(opp):
    '''
    if opp is not np.nan(float) then 
    take then take last element, if
    it exists else return np.nan (game
    hasnt happened as off data scrape).
    return last element.
    '''
    if type(opp) != float:
        infolist = opp_split(opp)
        if len(infolist) == 2:
            return np.nan
        else:
            return infolist[-1]
    else:
        return opp

In [19]:
fpl['opposition'] = fpl['opp'].apply(opp_extract_opposition)
fpl['is_home'] = fpl['opp'].apply(opp_extract_is_home)
fpl['score'] = fpl['opp'].apply(opp_extract_score)

Fixtures and game history are in the same column and use different names for teams. SOU becomes Southampton in fixtures.

In [20]:
names = {
    'Southampton':'SOU',
    'Bournemouth':'BOU',
    'WestHam':'WHU',
    'Cardiff':'CAR',
    'Everton':'EVE',
    'ManUtd':'MUN',
    'Chelsea':'CHE',
    'Brighton':'BHA',
    'Spurs':'TOT',
    'Huddersfield':'HUD',
    'Newcastle':'NEW',
    'Wolves':'WOL',
    'Arsenal':'ARS',
    'ManCity':'MCI',
    'Liverpool':'LIV',
    'CrystalPalace':'CRY',
    'Fulham':'FUL',
    'Leicester':'LEI',
    'Burnley':'BUR',
    'Watford':'WAT'
}
def map_names(oppo):
    if oppo in names.keys():
        oppo = names[oppo]
    return oppo

In [21]:
fpl['opposition'] = fpl['opposition'].apply(map_names)

Getting the score diff

In [22]:
def score_diff(score):
    if type(score) == float:
        return score
    else:
        return int(score[0]) - int(score[-1])

In [23]:
fpl['score_diff'] = fpl['score'].apply(score_diff)

Now that all information is properly stored there is no need for the original opp column.

In [24]:
fpl.drop('opp', axis = 1, inplace = True)

In [25]:
fpl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18641 entries, 0 to 37
Data columns (total 32 columns):
gw                    18641 non-null float64
player                18641 non-null object
team                  18641 non-null object
position              18641 non-null object
form                  18641 non-null float64
report                3368 non-null object
pts                   17057 non-null float64
mins                  17057 non-null float64
goals                 17057 non-null float64
assists               17057 non-null float64
clean                 17057 non-null float64
conceded              17057 non-null float64
own_goals             17057 non-null float64
pen_saved             17057 non-null float64
pen_missed            17057 non-null float64
yellows               17057 non-null float64
red                   17057 non-null float64
saves                 17057 non-null float64
bonus                 17057 non-null float64
bonus_sys             17057 non-null float64

no missing in gw, player, team, position, form, opposition and is_home. However, form is repeated over every player rather than the recorded changes in form it is just the last recorded form. Report is the injury status of players when they were scrapped, if players were not injured when scraped then report is missing. Report is also constant over the players rows rather than changing. Missing values in fixture_difficulty are for rows were the game hasnt occured yet as of the scrape, taken in week 34.

All other columns have missing values were games havnt orrcured yet, i.e. after week 34.

Can now merge the FPL data with the GIT data from the same season. Before that though the rows need to grouped by rounds, becuase of double game weeks some times players get two rows where they should only get one, both rows describing different games in the double game week.

gw, player, team, position, net_transfers and value should all remain constand when combining games in the same gameweek. opp should be dummified before the games are combined, then the row will have two 1s rather than one 1 for that gameweek. bonus_sys is between -100, 100 (assumed) and so should remain a negative/positive %. A new two week dummy should be created 1 indicating that the player has a double game week. All other columns can be summed. 

It could be argued that mins and saves should not be summed but a mean taken, or seperated into two columns for first and second game. This is because they are not strictly linearly related as they have thresholds. A gk with 2 saves in one game and 1 in another will get 0 points for both games, while a gk with 3 in one game and 0 in the next will get a point for saves. However, i dont believe adding columns will help the model. For thouroughness I create a column of meaned mins and saves, and anther for summed, and will test models to see which is better, i dont expect to see any meaningfull difference. 

In [26]:
#easy to sum along columns when grouping
fpl = pd.get_dummies(fpl, columns = ['opposition'])
git = pd.get_dummies(git, columns = ['opponent_team'])

In [27]:
#in order to later create a dummy if row is double game week
git['is_double'] = 0
fpl['is_double'] = 0

In [28]:
def is_double(alist):
    '''
    if a group is made then this returns 1
    '''
    the_list = [e for e in alist]
    if len(the_list) > 1:
        return 1
    else:
        return 0

def concater(alist):
    '''
    joins to strings when grouping them
    '''
    return ' & '.join([str(e) for e in alist])

def constantinople(alist):
    '''
    takes the first value (used for constants over weeks like value or position)
    '''
    return [e for e in alist][0]

def home_away(alist):
    '''
    returns a list of is_home values if 
    double week. This can later be delt with
    consistently over all seasons
    '''
    the_value = [e for e in alist]
    if len(the_value) == 1:
        return the_value[0]
    else:
        return the_value

In [29]:
def create_agg_git(df, mean_alt):
    '''
    creates the agg dict for the git groupby
    '''
    sum_me = ['assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus','clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt','fouls', 'goals_conceded', 'offside', 'open_play_crosses',
       'goals_scored','influence', 'key_passes','own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards','tackled',
       'tackles', 'target_missed','threat',
       'total_points','winning_goals', 'yellow_cards',
        'opponent_team_1', 'opponent_team_2', 'opponent_team_3',
       'opponent_team_4', 'opponent_team_5', 'opponent_team_6',
       'opponent_team_7', 'opponent_team_8', 'opponent_team_9',
       'opponent_team_10', 'opponent_team_11', 'opponent_team_12',
       'opponent_team_13', 'opponent_team_14', 'opponent_team_15',
       'opponent_team_16', 'opponent_team_17', 'opponent_team_18',
       'opponent_team_19', 'opponent_team_20']
    
    mean_me_alt = ['minutes', 'saves']
    sum_me_alt = ['minutes', 'saves']

    mean_me = ['bps', 'ea_index', 'ict_index'] #idk what eaindex is

    constant = ['id', 'player', 'transfers_balance', 'transfers_in', 'transfers_out','value', 'element', 'selected', 'loaned_in', 'loaned_out', 'round']

    concat = ['fixture','kickoff_time', 'kickoff_time_formatted']

    other = ['team_a_score', 'team_h_score','was_home'] 
    agg = {}
    if mean_alt == False:
        sum_me += sum_me_alt
    else:
        mean_me += mean_me_alt
        
    for col in df.columns:
        if col in sum_me:
            agg[col] = 'sum'
        elif col in mean_me:
            agg[col] = 'mean'
        elif col in constant:
            agg[col] = constantinople
        elif col in concat:
            agg[col] = concater
        elif col == 'is_double':
            agg[col] = is_double
        else:
            agg[col] = home_away
    
    return agg

def create_agg_fpl(df, mean_alt):
    '''
    creates the agg dict for the fpl groupby
    '''
    sum_me = ['pts', 'goals', 'assists', 'clean', 'conceded', 'own_goals', 'pen_saved',
        'pen_missed', 'yellows', 'red', 'bonus','influence', 'creativity', 'threat', 
        'net_transfers', 'opposition_ARS', 'opposition_BHA',
        'opposition_BOU', 'opposition_BUR', 'opposition_CAR', 'opposition_CHE','opposition_CRY', 
        'opposition_EVE', 'opposition_FUL', 'opposition_HUD', 'opposition_LEI', 'opposition_LIV', 
        'opposition_MCI', 'opposition_MUN','opposition_NEW', 'opposition_SOU', 'opposition_TOT', 
        'opposition_WAT', 'opposition_WHU', 'opposition_WOL']
    mean_me_alt = ['mins', 'saves', 'score_diff']
    sum_me_alt = ['mins', 'saves', 'score_diff']
    
    mean_me = ['fixture_difficulty', 'bonus_sys', 'ict_index']
    
    constant = ['player', 'team', 'position', 'form', 'report', 'gw', 'selected_by', 'value']
    
    concat = ['score']
    other = ['is_home']#for clarity rather than function
    
    agg = {}
    if mean_alt == False:
        sum_me += sum_me_alt
    else:
        mean_me += mean_me_alt
    
    for col in df.columns:
        if col in sum_me:
            agg[col] = 'sum'
        elif col in mean_me:
            agg[col] = 'mean'
        elif col in constant:
            agg[col] = constantinople
        elif col in concat:
            agg[col] = concater
        elif col in other:
            agg[col] = home_away
        elif col == 'is_double':
            agg[col] = is_double
    return agg

In [30]:
agg_fpl_mean = create_agg_fpl(fpl, mean_alt = True)
agg_fpl_sum = create_agg_fpl(fpl, mean_alt = False)
agg_git_mean = create_agg_git(git, mean_alt = True)
agg_git_sum = create_agg_git(git, mean_alt = False)

In [31]:
fpl_m = fpl.groupby(['player', 'gw']).agg(agg_fpl_mean)
fpl_s = fpl.groupby(['player', 'gw']).agg(agg_fpl_sum)
git_m = git.groupby(['player', 'round']).agg(agg_git_mean)
git_s = git.groupby(['player', 'round']).agg(agg_git_sum)

Now need to match the indexes of the two dataframes, and lowercase the player names in the FPL dataset.

In [32]:
fpl_m['player'] = fpl_m['player'].apply(lambda x: x.lower())

fpl_m.index = [fpl_m['player'], fpl_m['gw']]
git_m.index = [git_m['player'], git_m['round']]

fpl_m = fpl_m[fpl_m['gw']<35]

Datasets are now ready to be merged. 

In [33]:
df3 = git_m.merge(fpl_m, how='outer', left_on=git_m.index, right_on=fpl_m.index)
df3.columns = [col.replace('_x', '') for col in df3.columns]

This dataset is now ready to be merged with the other seasons datasets from git, once 
they too are cleaned.

In [34]:
df1 = pd.read_csv('../part-01/fpl_16_17.csv')
df2 = pd.read_csv('../part-01/fpl_17_18.csv')

In [35]:
df1.drop('fixture', axis=1, inplace=True)
df2.drop('fixture', axis=1, inplace=True)
df3.drop('fixture', axis=1, inplace=True)

In [36]:
teams_2016 = ['ARS', 'BOU', 'BUR', 'CHE', 'CRY', 'EVE', 'HUL', 'LEI',
             'LIV', 'MCI', 'MUN', 'MID', 'SOU', 'STO', 'SUN', 'SWA',
             'TOT', 'WAT', 'WBA', 'WHU']

teams_2017 = ['ARS', 'BOU', 'BHA', 'BUR', 'CHE', 'CRY', 'EVE', 'HUD',
             'LEI', 'LIV', 'MCI', 'MUN', 'NEW', 'SOU', 'STO', 'SWA',
             'TOT', 'WAT', 'WBA', 'WHU']

teams_2018 = ['ARS', 'BOU', 'BHA', 'BUR', 'CAR', 'CHE', 'CRY', 'EVE',
             'FUL', 'HUD', 'LEI', 'LIV', 'MCI', 'MUN', 'NEW', 'SOU',
             'TOT', 'WAT', 'WHU', 'WOL']

team_key_2016 = {'laurent koscielny':'ARS',
                'simon francis':'BOU',
                'tom heaton':'BUR',
                'john terry':'CHE',
                'scott dann':'CRY',
                'phil jagielka':'EVE',
                'michael dawson':'HUL',
                'wes morgan':'LEI',
                'jordan henderson':'LIV',
                'vincent kompany':'MCI',
                'wayne rooney':'MUN',
                'grant leadbitter':'MID',
                'steven davis':'SOU',
                'ryan shawcross':'STO',
                "john o'shea":'SUN',
                'leon britton':'SWA',
                'hugo lloris':'TOT',
                'troy deeney':'WAT',
                'darren fletcher':'WBA',
                'mark noble':'WHU'}

team_key_2017 = {'per mertesacker':'ARS',
                'simon francis':'BOU',
                'bruno saltor grau':'BHA',
                'tom heaton':'BUR',
                'gary cahill':'CHE',
                'jason puncheon':'CRY',
                'phil jagielka':'EVE',
                'tommy smith':'HUD',
                'wes morgan':'LEI',
                'jordan henderson':'LIV',
                'vincent kompany':'MCI',
                'michael carrick':'MUN',
                'jamaal lascelles':'NEW',
                'steven davis':'SOU',
                'ryan shawcross':'STO',
                'angel rangel':'SWA',
                'hugo lloris':'TOT',
                'troy deeney':'WAT',
                'jonny evans':'WBA',
                'mark noble':'WHU'}

team_key_2018 = {
                'laurent koscielny':'ARS',
                'simon francis':'BOU',
                'bruno saltor grau':'BHA',
                'tom heaton':'BUR',
                'sean morrison':'CAR',
                'gary cahill':'CHE',
                'luka milivojevic':'CRY',
                'phil jagielka':'EVE',
                'tom cairney':'FUL',
                'tommy smith':'HUD',
                'wes morgan':'LEI',
                'jordan henderson':'LIV',
                'vincent kompany':'MCI',
                'antonio valencia':'MUN',
                'jamaal lascelles':'NEW',
                'pierre-emile højbjerg':'SOU',
                'hugo lloris':'TOT',
                'troy deeney':'WAT',
                'mark noble':'WHU',
                'conor coady':'WOL'
}

num_team_2016 = {
    1:'ARS',
    2:'BOU',
    3:'BUR',
    4:'CHE',
    5:'CRY',
    6:'EVE',
    7:'HUL',
    8:'LEI',
    9:'LIV',
    10:'MCI',
    11:'MUN',
    12:'MID',
    13:'SOU',
    14:'STO',
    15:'SUN',
    16:'SWA',
    17:'TOT',
    18:'WAT',
    19:'WBA',
    20:'WHU'
}

num_team_2017 = {
    1:'ARS',
    2:'BOU',
    3:'BHA',
    4:'BUR',
    5:'CHE',
    6:'CRY',
    7:'EVE',
    8:'HUD',
    9:'LEI',
    10:'LIV',
    11:'MCI',
    12:'MUN',
    13:'NEW',
    14:'SOU',
    15:'STO',
    16:'SWA',
    17:'TOT',
    18:'WAT',
    19:'WBA',
    20:'WHU' 
}

num_team_2018 = {
    1:'ARS',
    2:'BOU',
    3:'BHA',
    4:'BUR',
    5:'CAR',
    6:'CHE',
    7:'CRY',
    8:'EVE',
    9:'FUL',
    10:'HUD',
    11:'LEI',
    12:'LIV',
    13:'MCI',
    14:'MUN',
    15:'NEW',
    16:'SOU',
    17:'TOT',
    18:'WAT',
    19:'WHU',
    20:'WOL'
}

Need to identify the double game weeks in the git datasets.

In [37]:
p_r_c_2016 = {}
for player in df1['player'].unique():
    player_round_counts = df1[(df1['player']==player)]['round'].value_counts()
    p_r_c_2016[player] = player_round_counts

p_r_c_2017 = {}
for player in df2['player'].unique():
    player_round_counts = df2[(df2['player']==player)]['round'].value_counts()
    p_r_c_2017[player] = player_round_counts
    
def is_double(row, prc):
    player_rounds = prc[row['player']]
    if player_rounds[row['round']]!=1:
        return 1
    else:
        return 0
    
def is_second_make(row, df):
    '''
    if row has a value count != 1
    and prev_gw==curr_gw, then this
    row is the second game in a double 
    game week.
    '''
    i = row.name
    is_double = row['is_double']
    gw = row['round']
    player = row['player']
    if is_double==1:
        prev_dub = df.loc[i-1,'is_double']
        prev_gw = df.loc[i-1, 'round']
        prev_player = df.loc[i-1, 'player']
        if (prev_dub==1) and (prev_gw==gw) and (prev_player==player):
            return 1
        else:
            return 0
    else:
        return 0

In [38]:
df1['is_double']=df1.apply(is_double, prc=p_r_c_2016, axis=1)
df2['is_double']=df2.apply(is_double, prc=p_r_c_2017, axis=1)

df1['is_second']=df1.apply(is_second_make, df=df1, axis = 1)
df2['is_second']=df2.apply(is_second_make, df=df2, axis = 1)

Players move between teams during transfers. This creates problems in both types of dataset. In the fpl-git 2018/19 dataset we observe teams but not the change as players from the fpl official site do not record past teams only current teams.

In the git 2017/18 and 2016/17 datasets we do not observe teams so need some way of identifying both the team and the team movements of players.

During each season a team has an official captain that will have remained with the team for at least that entire season. I find each captains fixtures and compare players to this. If they play the same team as a captain then they are in the same team as the captain. This is not robust to double weeks. For the 2016/17 and 2017/18 seasons this doesnt matter as rows are not yet collapsed. 

In [39]:
def cap_fixtures(df, team_key):
    '''
    get the features of the captain
    for each team, thereby getting
    the complete features of each team.
    This can later be used to identify the
    team of players.
    '''
    cap_opponents = {}
    for cap in team_key.keys():
        is_second=False
        opps=[]
        for gw in df[df['player']==cap]['round'].values:
            count = np.count_nonzero(df[df['player']==cap]['round'].values==gw)
            if count != 1:
                is_double=True
            else:
                is_double=False
            if is_double==False:
                opp = df[(df['player']==cap)&(df['round']==gw)]['opponent_team'].values[0]
            elif (is_double==True)&(is_second==True):
                opp = df[(df['player']==cap)&(df['round']==gw)]['opponent_team'].values[1]
                is_second=False
            elif (is_double==True)&(is_second==False):
                opp = df[(df['player']==cap)&(df['round']==gw)]['opponent_team'].values[0]
                is_second=True

            opps.append((gw,opp))
        cap_opponents[cap]=opps
    return cap_opponents

cap_opponents_1 = cap_fixtures(df1, team_key_2016)
cap_opponents_2 = cap_fixtures(df2, team_key_2017)

In [40]:
def team_assigner(row, cap_opponents, team_key):
    '''
    for each row finds the captain(s) that are
    playing the same opponent that week. As we
    know the team the cap plays we can normally
    deduce the team the row player plays for. This
    is not true in double weeks for the opponent
    were they will play two different caps from different
    teams and so the team deduction for the row is
    non-unique. If no captain is uniquely identified
    then np.nan is returned, else the deduced team
    of the row player.
    '''
    gw = row['round']
    opp = row['opponent_team']
    is_double = row['is_double']
    is_second = row['is_second']
    cap_matches = []
    valid = False
    while not valid:
        for cap in cap_opponents.keys():
            if gw not in [y for y,z in cap_opponents[cap]]:
                continue

            indicies = [i for i, x in enumerate([y for y,z in cap_opponents[cap]]) if x == gw]
            if (is_double==1) and len(indicies)!=2:
                continue
            elif (is_double==1) and (is_second==0):
                cap_opp = cap_opponents[cap][indicies[0]]
            elif (is_double==1) and (is_second==1):
                cap_opp = cap_opponents[cap][indicies[1]]
            elif is_double==0:
                cap_opp = cap_opponents[cap][indicies[0]]

            if cap_opp[1] == opp:
                cap_matches.append(team_key[cap])
        if len(cap_matches)==1:
            matched_team = cap_matches[0]
            valid=True
        else:
            return np.nan
    return matched_team

In [41]:
df1['team']=df1.apply(team_assigner, cap_opponents = cap_opponents_1, team_key = team_key_2016, axis = 1)
df2['team']=df2.apply(team_assigner, cap_opponents = cap_opponents_2, team_key = team_key_2017, axis = 1)

In [42]:
def fill_team(row, df):
    '''
    backward and
    forward filling team
    data to fill in missing team
    data when cap is non-unique.
    requires function as ffill cannot
    occur between players only within
    players. Leaves missing when adjacent
    player is different to row player.
    '''
    i = row.name
    player = row['player']
    if pd.isnull(row['team'])==True:
        #backward filler
        if player==df.loc[i-1,'player']:
            return df.loc[i-1, 'team']
        #forward filler
        elif player==df.loc[i+1,'player']:
            return df.loc[i+1, 'team']
        else:
            return np.nan
    else:
        return row['team']

In [43]:
df1['team']=df1.apply(fill_team, df=df1, axis=1)
df2['team']=df2.apply(fill_team, df=df2, axis=1)

In [44]:
df1['opponent_team']=df1['opponent_team'].apply(lambda x: num_team_2016[x])
df2['opponent_team']=df2['opponent_team'].apply(lambda x: num_team_2017[x])

df1 and df2 are now clean in team columns, but df3 still needs some work. Done sepereratly becuase the merge has already been made and so double game weeks need to be treated differently.

Dummy names for opposition need to be renamed for coding convienence.

There are currently two sets of dummies in df3 that inform the row on the opponent teams that week, one derived from the github database and one from the FPL scrape. The fpl scrape is missing info on players that left the premier league before the scrape. The github data has numbered them alphabeticaly, i.e. ARS is opponent_team_1. So, I drop the fpl opponent data and rename the github columns to make more sence.

In [50]:
opps = [col for col in df3.columns if 'opponent_team' in col]

In [51]:
def add_full_name(name, intro):
    return intro+str(name)

rename_dict = {}
for key in num_team_2018.keys():
    rename_dict[
        add_full_name(name=key,
                      intro='opponent_team_')
               ]=num_team_2018[key]

In [52]:
old_dummies=list([add_full_name(col, 'opposition_') for col in rename_dict.values()])

df3.drop(old_dummies,axis=1,inplace=True)

df3.rename(columns=rename_dict, inplace = True)

In [53]:
def cap_fixtures_3(df, team_key):
    '''
    Works similarly to cap_fixtures function.
    Differences come from the opponent team already
    being dummied and unique gameweeks.
    '''
    cap_opponents = {}
    poss_opp = ['ARS', 'BOU', 'BHA', 'BUR', 'CAR', 'CHE', 'CRY', 'EVE', 'FUL',
       'HUD', 'LEI', 'LIV', 'MCI', 'MUN', 'NEW', 'SOU', 'TOT', 'WAT', 'WHU',
       'WOL']
    for cap in team_key.keys():
        #cap = team_key[team]
        is_second=False
        opps=[]
        for gw in df[df['player']==cap]['round'].values:
            #all unique due to merge
            tmp = df[(df['player']==cap)&(df['round']==gw)]
            if tmp['is_double'].values[0]==1:
                #double week
                dopps=[gw]
                for o in poss_opp:
                    if tmp[o].values[0]==1:
                        dopps.append(o)
                dopps = tuple(dopps)
                opps.append(dopps)
                
            elif tmp['is_double'].values[0]==0:
                #single week
                for o in poss_opp:
                    if tmp[o].values[0]==1:
                        opps.append((gw,o))
                        break
        cap_opponents[cap]=opps
    return cap_opponents

In [54]:
cap_opponents_3 = cap_fixtures_3(df3, team_key_2018)

In [55]:
def team_assigner_3(row, cap_opponents, team_key):
    '''
    matches exactly with captains. Is only
    ambiguos when two teams with double
    weeks play the same opponents, this never
    happens so all good. Overrights original
    team data as this is more accurate, original
    data was playerwide stat and not measured over
    time. So when the player changed team this wasnt
    accounted for. This method of matching them with
    captains accounts for player movements between teams.
    '''
    gw = row['round']
    poss_opp = ['ARS', 'BOU', 'BHA', 'BUR', 'CAR', 'CHE', 'CRY', 'EVE', 'FUL',
       'HUD', 'LEI', 'LIV', 'MCI', 'MUN', 'NEW', 'SOU', 'TOT', 'WAT', 'WHU',
       'WOL']
    opps=[gw]
    for o in poss_opp:
        if row[o]==1:
            opps.append(o)
    opps = tuple(opps)
    for cap in cap_opponents.keys():
        cap_gws = [tup[0] for tup in cap_opponents[cap]]
        if gw in cap_gws:
            i = cap_gws.index(gw)
            opp_to_match = cap_opponents[cap][i]
            if opps == opp_to_match:
                team = team_key[cap]
                return team

In [56]:
df3['team1']=df3.apply(team_assigner_3, cap_opponents=cap_opponents_3, team_key=team_key_2018, axis=1)

In [57]:
df3['team']=df3['team1']

df1, df2 and df3 have complete team data now. They are still missing position data. 


For df1 and df2 player positions known in df3 can be matched and assigned. When player is not in df3 the only way to know their position is to infer it with the points they get for actions within game. E.g. midfielders get 5 points for a goal while forwards get 4, defenders and goalkeepers get 6. Therefore, if the player gets a goal we can infer their positon from the total points they get, for defenders and goalkeepers we can distinguish by goals becuase no goalkeeper has scoared in the last 3 seasons outside a penalty (https://sabotagetimes.com/sport/the-5-greatest-premiership-goals-scored-by-keepers). This is 1 of 4 freatures we can use to identify positions. We can identify by using:

1) goals

2) saves

3) goals conceded

4) clean sheets

df3 cannot use this inference on double weeks as saves and goals conceded add/subtract less than one point within a game but not between games. It takes 3 saves to gain a point and 2 goals concende to lose a point for goalkeepers within a game.

I do assume that the rules for points hasnt changed over the last three years.

If a player identifies their position in one game throughtout their entire observed career all missing positonal data is filled with the identified position. This is safe as FPL keeps constant positions for players no matter what position they actually play in a game.

In [58]:
position_dict = {}
for player in df3['player'].unique():
    pos = df3[df3['player']==player]['position'].unique()[0]
    position_dict[player]=pos

In [59]:
def position_fill(row, position_dict):
    if row['player'] in position_dict.keys():
        return position_dict[row['player']]
    else:
        return 'unkown'

In [60]:
df1['position']=df1.apply(position_fill, position_dict=position_dict, axis=1)
df2['position']=df2.apply(position_fill, position_dict=position_dict, axis=1)

In [61]:
def calculate_points(row, position):
    points = 0
    if row['minutes']>=60:
        points += 2 #playing more than 60
    elif (row['minutes']>0) and (row['minutes']<60):
        points += 1 #playing up to 60 mins
    points += row['assists']*3 #assists
    points += row['penalties_saved']*5 #penalties saved
    points += row['bonus']*1 #bonus points
    points -= row['penalties_missed']*2 #penalties missed
    points -= row['yellow_cards']*1 #yellow card
    points -= row['red_cards']*3 #red card
    points -= row['own_goals']*2 #own goals
    if position == 'Goalkeeper':
        points += row['goals_scored']*6 #goals scored
        points += row['clean_sheets']*4 #clean sheet
        points += int(row['saves']*(1/3)) #saves made
        points -= int(row['goals_conceded']*(1/2)) #goals conceded
    elif position == 'Defender':
        points += row['goals_scored']*6 #goals scored
        points += row['clean_sheets']*4 #clean sheet
        points -= int(row['goals_conceded']*(1/2)) #goals conceded
    elif position == 'Midfielder':
        points += row['goals_scored']*5 #goals scored
        points += row['clean_sheets']*1 #clean sheet
    elif position == 'Forward': #Forward
        points += row['goals_scored']*4 #goals scored
    return points

def position_assignment(row):
    positions = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
    points = [calculate_points(row, position) for position in positions]
    check = []
    for p in points:
        if p == row['total_points']:
            check.append(True)
        else:
            check.append(False)
            
    if check.count(True) == 1: #only return non-ambiguos positions
        #correction, defenders and gk are ambiguos when no saves 
        #are made. They are dealt with later in fill_p_pos().
        i = check.index(True)
        return positions[i]
    else:
        return 'unkown'

def position_safe_ovwr(row, safe_double=True):
    if (safe_double==False) and (row['is_double']==1):
        return row['position']
    infer_pos = position_assignment(row)
    if (row['position']=='unkown') or pd.isnull(row['position']):
        return infer_pos
    else:
        return row['position']

In [62]:
df1['position']=df1.apply(position_safe_ovwr, axis = 1)
df2['position']=df2.apply(position_safe_ovwr, axis = 1)
df3['position']=df3.apply(position_safe_ovwr, safe_double = False, axis = 1)

In [63]:
def fill_p_pos(df):
    player_positions = {}
    for player in df['player'].unique():
        tmp = df[df['player']==player]
        pos_set = set(tmp['position'].values)
        #if results will be unkown and some other pos:
        if len(pos_set)==2:
            if 'unkown' not in pos_set:
                print(player, pos_set)
            elif 'Goalkeeper' in pos_set:
                player_positions[player]='Goalkeeper'
            elif 'Defender' in pos_set:
                player_positions[player]='Defender'
            elif 'Midfielder' in pos_set:
                player_positions[player]='Midfielder'
            else: #Forward
                player_positions[player]='Forward'
                
        #should not occur there to test
        elif len(pos_set)>3:
            print(player, pos_set)
        #no unkowns player position was not 
        #infered or they distinguished themselves
        #everygame
        else:
            player_positions[player]=pos_set.pop()
    return player_positions

In [64]:
pos_1 = fill_p_pos(df1)
pos_2 = fill_p_pos(df2)
pos_3 = fill_p_pos(df3)

The dictionaries pos_# contain the position of a player if known or the position infered if unkown and so will not overwrite player position when applied.

In [65]:
df1['position']=df1.apply(lambda x: pos_1[x['player']], axis=1)
df2['position']=df2.apply(lambda x: pos_2[x['player']], axis=1)
df3['position']=df3.apply(lambda x: pos_3[x['player']], axis=1)

Positional data is now present in all dataframes.

Now comes a little cleaning/feature engineering on team scores and leage points. Currently df3 has score (as a string) and the goal difference of a game (team - opponent). df1 and df2 only have the scores for the home and away team. All three dfs need to be cleaned to have scores for team and oppponent and goal difference. 

Further the league points of a team can be calculated, where a lose = 0, draw = 1 and win = 3. This is cumulative over the season and captains will be again used to caluculate the cumalitive points for the team.

Again df3 will need to be treated slightly differently for different cleaning goals, and different data structure (already grouped by weeks so double weeks are a concern)

In [66]:
def get_scores(row, team=True):
    home = row['was_home']
    if home==team:
        score = row['team_h_score']
    else:
        score = row['team_a_score']
    return score

In [67]:
df1['team_score']=df1.apply(get_scores, axis=1)
df1['opp_score']=df1.apply(get_scores, team=False, axis=1)
df2['team_score']=df2.apply(get_scores, axis=1)
df2['opp_score']=df2.apply(get_scores, team=False, axis=1)

In [68]:
def get_league_points(row):
    if row['team_score']>row['opp_score']:
        return 3
    elif row['team_score']==row['opp_score']:
        return 1
    elif row['team_score']<row['opp_score']:
        return 0

In [69]:
df1['league_points']=df1.apply(get_league_points,axis=1)
df2['league_points']=df2.apply(get_league_points,axis=1)

In [70]:
point_table_1 = {}
for cap in team_key_2016.keys():
    p_list = df1[df1['player']==cap]['league_points'].values
    gw_list = df1[df1['player']==cap]['round'].values
    point_table_1[team_key_2016[cap]]=list(zip(gw_list,np.cumsum(p_list)))

point_table_2 = {}
for cap in team_key_2017.keys():
    p_list = df2[df2['player']==cap]['league_points'].values
    gw_list = df2[df2['player']==cap]['round'].values
    
    point_table_2[team_key_2017[cap]]=list(zip(gw_list,np.cumsum(p_list)))

In [71]:
def cum_points(row, point_table):
    '''
    point tables are applied to players
    from the rows gw and team.
    '''
    gw = row['round']
    is_second = row['is_second']
    is_double = row['is_double']
    team = row['team']
    if is_double==0:
        i = [gw for gw,p in point_table[team]].index(gw)
        points = point_table[team][i]
        
    else:
        i = [i for i,x in enumerate(
            [gw for gw,p in point_table[team]]
        ) if x==gw] #find indexes of gws of row
        if is_second==0:
            points = point_table[team][i[0]]
        else:
            points = point_table[team][i[1]]
    return points[1]

In [72]:
df1['league_points_cum']=df1.apply(cum_points, point_table=point_table_1, axis=1)
df2['league_points_cum']=df2.apply(cum_points, point_table=point_table_2, axis=1)

In [73]:
def get_scores_3(row, df, team=True):
    '''
    using the same method as for df1 and df2
    but robust to double weeks.
    '''
    home=row['was_home']
    team_a=row['team_a_score']
    team_h=row['team_h_score']
    is_double=row['is_double']
    i = row.name
    if i != 0:
        #not a double game week in first
        #row so safe to filter out
        if df.loc[i-1, 'is_double']==1:
            prev_is_double=1
        elif df.loc[i-1, 'is_double']==0:
            prev_is_double=0
    if is_double==0:
        if home == team:
            score = team_h
        else:
            score = team_a
    else:
        if prev_is_double == 0:#get first game
            if home[0] == team:
                score = int(team_h[0])
            else:
                score = int(team_a[0])
        elif prev_is_double == 1:#get second game
            if home[1] == team:
                score = int(team_h[1])
            else:
                score = int(team_a[1])
    return score

In [74]:
df3['team_score']=df3.apply(get_scores_3, df=df3, axis=1)
df3['opp_score']=df3.apply(get_scores_3, df=df3, team=False, axis=1)

In [75]:
def outcome(team, opp):
    if team>opp:
        return 3
    elif team<opp:
        return 0
    else:
        return 1

def team_league_points_3(row, df):
    home=row['was_home']
    team_a=row['team_a_score']
    team_h=row['team_h_score']
    is_double=row['is_double']
    i=row.name
    if i != 0:
        if df.loc[i-1,'is_double']==1:
            prev_is_double=1
        elif df.loc[i-1,'is_double']==0:
            prev_is_double=0
    if is_double==0:
        if home==True:
            points = outcome(team_h,team_a)
        elif home==False:
            points = outcome(team_a,team_h)
    else:
        if prev_is_double==0:
            if home[0]==True:
                points = outcome(team_h[0],team_a[0])
            elif home[0]==False:
                points = outcome(team_a[0],team_h[0])
        elif prev_is_double==1:
            if home[1]==True:
                points = outcome(team_h[1],team_a[1])
            elif home[1]==False:
                points = outcome(team_a[1],team_h[1])
    return points

In [76]:
df3['league_points']=df3.apply(team_league_points_3,df=df3,axis=1)

In [77]:
point_table_3 = {}
for cap in team_key_2018.keys():
    p_list = df3[df3['player']==cap]['league_points'].values
    gw_list = df3[df3['player']==cap]['round'].values
    
    point_table_3[team_key_2018[cap]]=list(zip(gw_list,np.cumsum(p_list)))

In [78]:
def cum_points_3(row, point_table):
    #not entirely sure this is neccessery
    #not working
    gw = row['round']
    team = row['team']
    
    i = [r for r,p in point_table[team]].index(gw)
    
    points = point_table[team][i]
    
    return points[1]

In [79]:
df3['league_points_cum']=df3.apply(cum_points_3, point_table=point_table_3, axis=1)

Now that team data is cleaned, kickoff data is next. While the exact time of the kickoff is informative I will only be using a dummy for if late or not, if kickoff is after 6pm then the game is late (1) else early (0).

In [80]:
def is_late(kickoff):
    if int(kickoff[-5:-3])>18:
        return 1
    else:
        return 0

In [81]:
df1['is_late']=df1['kickoff_time_formatted'].apply(is_late)
df2['is_late']=df2['kickoff_time_formatted'].apply(is_late)

In [82]:
def is_late_3(row):
    if row['is_double']==1:
        kickoff = row['kickoff_time_formatted'][7:9]
        try:
            kickoff = int(kickoff)
        except:
            kickoff = int(kickoff[1:])
        if kickoff>18:
            return 1
        else:
            return 0
    elif row['is_double']==0:
        first = row['kickoff_time_formatted'][7:9]
        second = row['kickoff_time_formatted'][-12:-10]
        try:
            first = int(first)
        except:
            first = int(first[1:])
        try:
            second = int(second)
        except:
            second = int(second[1:])
        if (first>18)&(second>18):
            return 1
        else:
            return 0

In [83]:
df3['is_late']=df3.apply(is_late_3, axis=1)

Now, finally, I group 2016/17 and 2017/19 by gameweek so that each row represent a single week rather than a game.

In [84]:
to_drop=['Unnamed: 0', 'ea_index', 'element', 'id', 'kickoff_time',
        'kickoff_time_formatted', 'loaned_in', 'loaned_out',
        'team_a_score', 'team_h_score', 'is_second', 'league_points']

constants=['selected', 'transfers_balance', 'transfers_in', 'transfers_out',
          'value', 'player', 'is_double', 'team', 'position', 'round']
take_last = ['league_points_cum']
rules=['is_late', 'was_home']
to_mean = ['bps', 'minutes', 'saves']
to_dummy=['opponent_team']
to_sum=[col for col in df1.columns if col not in (to_drop+constants+rules+to_mean+to_dummy)]

In [85]:
pan1=pd.get_dummies(df1, columns=to_dummy, prefix='',prefix_sep='')
pan2=pd.get_dummies(df2, columns=to_dummy, prefix='',prefix_sep='')

In [86]:
def constant_make(alist):
    alist = [e for e in alist]
    return alist[0]

def is_late_group(alist):
    if 1 in alist:
        return 1
    else:
        return 0

def was_home_group(alist):
    if True in alist:
        return 1
    else:
        return 0
    
def create_agg(df, opp_teams, constants=constants, rules=rules, to_mean=to_mean, to_sum=to_sum):
    agg={}
    for constant in constants:
        agg[constant]=constant_make
    for mean_me in to_mean:
        agg[mean_me]='mean'
    for sum_me in to_sum+opp_teams:
        agg[sum_me]='sum'
    for rule in rules:
        if rule=='is_late':
            agg[rule]=is_late_group
        else:
            agg[rule]=was_home_group
    
    #league_points
    agg['league_points_cum']='max'
    return agg

In [87]:
agg_1 = create_agg(pan1, opp_teams=teams_2016)
agg_2 = create_agg(pan2, opp_teams=teams_2017)

In [88]:
pan1.drop(to_drop, axis = 1, inplace = True)
pan2.drop(to_drop, axis = 1, inplace = True)

In [89]:
pan1=pan1.groupby(['player', 'round']).agg(agg_1)
pan2=pan2.groupby(['player', 'round']).agg(agg_2)

In [90]:
df3_drop=['element','ea_index','id','kickoff_time','kickoff_time_formatted',
         'loaned_in','loaned_out','team_a_score','team_h_score','league_points',
         'order','team1']
pan3=df3.drop(df3_drop, axis=1)

LAST THING!!! The was_home dummy is still in True False format, it needs to be in 1 0 format.

In [91]:
def was_home_num(was_home):
    if was_home == False:
        return 0
    elif was_home == True:
        return 1
    else:
        return was_home #some are already num
    

pan1['was_home']=pan1['was_home'].apply(was_home_num)
pan2['was_home']=pan2['was_home'].apply(was_home_num)

In [92]:
def was_home_num_3(was_home):
    if type(was_home)==list:
        if True in was_home:
            return 1
        else:
            return 0
    else:
        if was_home==True:
            return 1
        else:
            return 0
pan3['was_home']=pan3['was_home'].apply(was_home_num_3)

In [102]:
to_drop=[col for col in pan3.columns if '_y' in col]
also_drop=[
    'gw',
    'pts',
    'mins',
    'goals',
    'clean',
    'conceded',
    'pen_saved',
    'pen_missed',
    'yellows',
    'red',
    'bonus_sys',
    'net_transfers',
    'selected_by',
    'fixture_difficulty',
    'is_home',
    'score'
]
pan3.drop(to_drop+also_drop, axis=1, inplace=True)

In [104]:
pan1.to_csv('cleaned_pan1.csv')
pan2.to_csv('cleaned_pan2.csv')
pan3.to_csv('cleaned_pan3.csv')

In [103]:
list(pan3.columns)

['key_0',
 'Unnamed: 0',
 'assists',
 'attempted_passes',
 'big_chances_created',
 'big_chances_missed',
 'bonus',
 'bps',
 'clean_sheets',
 'clearances_blocks_interceptions',
 'completed_passes',
 'creativity',
 'dribbles',
 'errors_leading_to_goal',
 'errors_leading_to_goal_attempt',
 'fouls',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'key_passes',
 'minutes',
 'offside',
 'open_play_crosses',
 'own_goals',
 'penalties_conceded',
 'penalties_missed',
 'penalties_saved',
 'recoveries',
 'red_cards',
 'round',
 'saves',
 'selected',
 'tackled',
 'tackles',
 'target_missed',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'was_home',
 'winning_goals',
 'yellow_cards',
 'player',
 'ARS',
 'BOU',
 'BHA',
 'BUR',
 'CAR',
 'CHE',
 'CRY',
 'EVE',
 'FUL',
 'HUD',
 'LEI',
 'LIV',
 'MCI',
 'MUN',
 'NEW',
 'SOU',
 'TOT',
 'WAT',
 'WHU',
 'WOL',
 'is_double',
 'team',
 'position',
 'form',
 'report',
 'score_diff',
 'team_scor

In [98]:
pan3['player'].isnull().sum()

0